In [1]:
# SparkContext is already defined as sc
HDFS = 'hdfs://scut0:9000/bike/'

# Extracting the right features from data

In [2]:
dataFile = sc.textFile(HDFS + 'hour_noheader.csv')
data = dataFile.map(lambda line : line.strip().split(','))
print(data.count())
print(data.first())
data.cache()

17379
[u'1', u'2011-01-01', u'1', u'0', u'1', u'0', u'0', u'6', u'0', u'1', u'0.24', u'0.2879', u'0.81', u'0', u'3', u'13', u'16']


PythonRDD[4] at RDD at PythonRDD.scala:48

In [3]:
# one-hot encoding for feature[2:10]
length = sum([data.map(lambda x:x[i]).distinct().count() for i in xrange(2,10)])
print('total length of categorical feature:{0}'.format(length))

def get_mapping(rdd, idx):
    return rdd.map(lambda x:x[idx]).distinct().zipWithIndex().collectAsMap()

mappings = [get_mapping(data, i) for i in xrange(2,10)]

cat_len = sum([len(m) for m in mappings])
num_len = len(data.first()[11:15])
total_len = cat_len + num_len

print("Feature vector length for categorical features: %d" % cat_len)
print("Feature vector length for numerical features: %d" % num_len)
print("Total feature vector length: %d" % total_len)

total length of categorical feature:57
Feature vector length for categorical features: 57
Feature vector length for numerical features: 4
Total feature vector length: 61


## Creating feature vectors for the linear model

In [4]:
import numpy as np
from pyspark.mllib.regression import LabeledPoint

# extract features and label and encapsulate as LabeledPoint
def extract_features(record, mappings, cat_len):
    cate_features = np.zeros(cat_len)
    offset = 0
    for i, val in enumerate(record[2:10]):
        idx = offset + mappings[i][val]
        cate_features[idx] = 1
        offset += len(mappings[i])
    num_features = np.array([float(field) for field in record[10:14]])
    return np.concatenate((cate_features, num_features))

print(extract_features(data.first(), mappings, cat_len))

def extract_label(record):
    return float(record[-1])

processed_data = data.map(lambda x: LabeledPoint(extract_label(x), extract_features(x,mappings, cat_len)))
print(processed_data.first())

[ 1.      0.      0.      0.      0.      1.      0.      1.      0.      0.
  0.      0.      0.      0.      0.      0.      0.      0.      0.      0.
  0.      0.      0.      0.      0.      0.      0.      0.      0.      0.
  0.      0.      0.      0.      0.      0.      1.      0.      0.      0.
  0.      0.      0.      1.      0.      0.      0.      0.      0.      0.
  1.      0.      1.      1.      0.      0.      0.      0.24    0.2879
  0.81    0.    ]
(16.0,[1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.24,0.2879,0.81,0.0])


## Creating feature vectors for the decision tree

In [5]:
# decision tree models typically work on raw features 
# that is, it is not required to convert categorical features into a binary vector encoding; 
def extract_features_dt(record):
    return np.array(map(float, record[2:14]))
dt_data = data.map(lambda x:LabeledPoint(extract_label(x), extract_features_dt(x)))
print(dt_data.first())

(16.0,[1.0,0.0,1.0,0.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.81,0.0])


# Training and using regression models

In [6]:
from pyspark.mllib.regression import LinearRegressionWithSGD
from pyspark.mllib.tree import DecisionTree

## Train  model

In [7]:
help(LinearRegression)

Name: org.apache.toree.interpreter.broker.BrokerException
Message: Traceback (most recent call last):
  File "/tmp/kernel-PySpark-08b9ceba-bf5a-429b-9f45-231454d0ee26/pyspark_runner.py", line 194, in <module>
    eval(compiled_code)
  File "<string>", line 1, in <module>
NameError: name 'LinearRegression' is not defined

StackTrace: org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:163)
org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:163)
scala.Option.foreach(Option.scala:257)
org.apache.toree.interpreter.broker.BrokerState.markFailure(BrokerState.scala:162)
sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
java.lang.reflect.Method.invoke(Method.java:498)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
py4j.refl

In [8]:
help(DecisionTree.trainRegressor)

Help on method trainRegressor in module pyspark.mllib.tree:

trainRegressor(cls, data, categoricalFeaturesInfo, impurity='variance', maxDepth=5, maxBins=32, minInstancesPerNode=1, minInfoGain=0.0) method of __builtin__.type instance
    Train a decision tree model for regression.
    
    :param data:
      Training data: RDD of LabeledPoint. Labels are real numbers.
    :param categoricalFeaturesInfo:
      Map storing arity of categorical features. An entry (n -> k)
      indicates that feature n is categorical with k categories
      indexed from 0: {0, 1, ..., k-1}.
    :param impurity:
      Criterion used for information gain calculation.
      The only supported value for regression is "variance".
      (default: "variance")
    :param maxDepth:
      Maximum depth of tree (e.g. depth 0 means 1 leaf node, depth 1
      means 1 internal node + 2 leaf nodes).
      (default: 5)
    :param maxBins:
      Number of bins used for finding splits at each node.
      (default: 32)
    :

In [9]:
# linear regression
linear_model = LinearRegressionWithSGD.train(processed_data, iterations=1000, step=0.1, intercept=False)
true_vs_predict = processed_data.map(lambda p: (p.label, linear_model.predict(p.features)))
print(true_vs_predict.first())

/opt/spark/python/pyspark/mllib/regression.py:281: UserWarning: Deprecated in 2.0.0. Use ml.regression.LinearRegression.
  warnings.warn("Deprecated in 2.0.0. Use ml.regression.LinearRegression.")
(16.0, 72.447168855043827)


In [10]:
# decision tree
dt_model = DecisionTree.trainRegressor(dt_data, {})
preds = dt_model.predict(dt_data.map(lambda x : x.features))
true_vs_predict_dt = dt_data.map(lambda p:p.label).zip(preds)
print(true_vs_predict_dt.first())

(16.0, 54.913223140495866)


## Evaluate models

In [11]:
def squared_error(record):
    return (record[0] - record[1])**2

def abs_error(record):
    return abs(record[0] - record[1])

mean_squared_error = true_vs_predict.map(lambda r:squared_error(r)).mean()
mean_abs_error = true_vs_predict.map(lambda r:abs_error(r)).mean()
print('Linear Regression:\nMean squared error:{0}\nMean absolute error:{1}'.format(mean_squared_error, mean_abs_error))

mean_squared_error_dt = true_vs_predict_dt.map(lambda r:squared_error(r)).mean()
mean_abs_error_dt = true_vs_predict_dt.map(lambda r:abs_error(r)).mean()
print('Decision Tree:\nMean squared error:{0}\nMean absolute error:{1}'.format(mean_squared_error_dt, mean_abs_error_dt))

Linear Regression:
Mean squared error:22905.9187726
Mean absolute error:114.67503594
Decision Tree:
Mean squared error:11611.4859995
Mean absolute error:71.1501878649


# Improving model performance and tuning parameters

## Transforming the target variable

In [12]:
# many machine learning models, including linear models
# make assumptions regarding the distribution of the input data as well as target variables.
# In particular, linear regression assumes a normal distribution

# when target variable do not follow a normal distribution
# take the logarithm of the target value instead of the raw value. 
# This is often referred to as log-transforming the target variable
log_transformed_data = processed_data.map(lambda p: LabeledPoint(np.log(p.label), p.features))


# A second type of transformation that is useful in the case of target values that do not
# take on negative values and, in addition, might take on a very wide range of values,
# is to take the square root of the variable
square_transformed_data = dt_data.map(lambda p: LabeledPoint(np.sqrt(p.label), p.features))

In [13]:
# performance of linear regression after log transform of target variable
linear_model_log =  LinearRegressionWithSGD.train(log_transformed_data, iterations=1000, step=0.1, intercept=False)
true_vs_predict_log = log_transformed_data.map(lambda p: (p.label, linear_model_log.predict(p.features)))
mean_squared_error_log = true_vs_predict_log.map(lambda r:squared_error(r)).mean()
mean_abs_error_log = true_vs_predict_log.map(lambda r:abs_error(r)).mean()
print('Linear Regression after log transform:\nMean squared error:{0}\nMean absolute error:{1}'.format(mean_squared_error_log, mean_abs_error_log))

Linear Regression after log transform:
Mean squared error:1.73824417498
Mean absolute error:1.03879865848


In [14]:
# performance of decision tree after log transform of target variable
dt_model_log = DecisionTree.trainRegressor(square_transformed_data, {})
preds = dt_model_log.predict(square_transformed_data.map(lambda x : x.features))
true_vs_predict_dt_log = square_transformed_data.map(lambda p:p.label).zip(preds)
mean_squared_error_dt_log = true_vs_predict_dt_log.map(lambda r:squared_error(r)).mean()
mean_abs_error_dt_log = true_vs_predict_dt_log.map(lambda r:abs_error(r)).mean()
print('Decision Tree after log transformation:\nMean squared error:{0}\nMean absolute error:{1}'.format(mean_squared_error_dt_log, mean_abs_error_dt_log))

Decision Tree after log transformation:
Mean squared error:11.1814998545
Mean absolute error:2.43810743015


# Tuning model parameters

## Creating training and testing sets to evaluate parameters

In [15]:
# Spark's Python API does not yet provide the randomSplit convenience method that is available in Scala
# Hence, we will need to create a training and test dataset manually

# We can achieve this using the sample method to take a random sample for our test set
# followed by using the subtractByKey method, which takes care of 
# returning the elements in one RDD where the keys do not overlap with the other RDD

# subtractByKey, as the name suggests, works on the keys of the RDD elements that consist of key-value pairs
# Therefore, here we will use zipWithIndex on our RDD of extracted training examples. 
# This creates an RDD of (LabeledPoint, index) pairs

test_data_ratio = 0.2
index_log_transformed_data = log_transformed_data.zipWithIndex().map(lambda (k,v) :(v, k))
test_data = index_log_transformed_data.sample(withReplacement = False, fraction = test_data_ratio, seed = 123)
train_data = index_log_transformed_data.subtractByKey(test_data)

train_data = train_data.map(lambda (idx, p): p)
test_data = test_data.map(lambda (idx, p):p)
print('number of train data: {0}\nnumber of test data: {1}'.format(train_data.count(), test_data.count()))

number of train data: 13915
number of test data: 3464


## The impact of parameter settings for linear models

In [16]:
def evaluate(train, test, iterations, step, regParam, regType, intercept):
    model = LinearRegressionWithSGD.train(train, iterations, step,\
                                          regParam=regParam, regType=regType, intercept=intercept)
    tp = test.map(lambda p: (p.label, model.predict(p.features)))
    mse = tp.map(lambda p: squared_error(p)).mean()
    return mse

In [17]:
# iterations
# store all different values needed to be evaluated in a list
params = [1, 5, 10, 20, 50, 100]
metrics = [evaluate(train_data, test_data, param, 0.01, 0.0, 'l2', False) for param in params]
print(params)
print(metrics)

[1, 5, 10, 20, 50, 100]
[20.845951702978386, 17.747592473707126, 15.645164342260253, 13.109441839169357, 9.3694050200682746, 6.6405468137387489]


In [18]:
# Step size
params = [0.01, 0.025, 0.05, 0.1, 1.0]
metrics = [evaluate(train_data, test_data, 10, param, 0.0, 'l2', False) for param in params]
print(params)
print(metrics)

[0.01, 0.025, 0.05, 0.1, 1.0]
[15.645164342260253, 9.3171259065620831, 4.4756827986761412, 2.2509450032709895, 1.4501825618154074]


In [19]:
# different levels of L2 regularization 
params = [0.0, 0.01, 0.1, 1.0, 5.0, 10.0, 20.0]
metrics = [evaluate(train_data, test_data, 10, 0.1, param, 'l2', False) for param in params]
print(params)
print(metrics)

[0.0, 0.01, 0.1, 1.0, 5.0, 10.0, 20.0]
[2.2509450032709895, 2.2573695890496688, 2.3189589425597323, 3.2151109211262123, 8.357292719160256, 12.46097422684992, 16.220273970916246]


In [20]:
# effect of adding an intercept term to the model
# always add the intercept
params = [False, True]
metrics = [evaluate(train_data, test_data, 10, 0.1, 1.0, 'l2', param) for param in params]
print(params)
print(metrics)

[False, True]
[3.2151109211262123, 2.6797426251036804]
